# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import pandas as pd
import random
import importlib
import time
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from config import Config
from normalizer import MeanStdNormalizer
from ppo_agent import PPOAgent
from network import GaussianActorCriticNet, FCBody


from helper import *
RANDOM_SEED = 42

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


When finished, you can close the environment.

In [6]:
# env.close()

### Agent

### ppo_agent.py

In [113]:
#######################################################################
# Copyright (C) 2017 Shangtong Zhang(zhangshangtong.cpp@gmail.com)    #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

# Modified by Jeff Daniels (jeffrey.arthur.daniels@gmail.com) 2020
# For Udacity Deep Reinforcement Learning Nanodegree

import numpy as np
import torch
from replay import Storage
from torch_utils import to_np, tensor
from misc_utils import random_sample 

class PPOAgent():
    def __init__(self, config):
        self.config = config
        self.environment = config.environment
        self.brain_name = config.brain_name
        self.network = config.network_fn()
        self.actor_opt = config.actor_opt_fn(self.network.actor_params)
        self.critic_opt = config.critic_opt_fn(self.network.critic_params)
        self.total_steps = 0
        self.states = self.environment.reset(train_mode=True)[self.brain_name].vector_observations
        self.states = config.state_normalizer(self.states)
        self.storage = Storage(config.rollout_length)
        self.dones = []
        self.rewards = 0


    def step(self):

        prediction = self.network(self.states)
        environment_info = self.environment.step(to_np(prediction['a']))[self.brain_name]
        next_states = environment_info.vector_observations 
        self.rewards = environment_info.rewards
        self.dones = environment_info.local_done
        rewards = self.config.reward_normalizer(self.rewards)
        next_states = self.config.state_normalizer(next_states)
        self.storage.add(prediction)
        self.storage.add({'r': tensor(rewards).unsqueeze(-1),
                          'm': tensor(dones).unsqueeze(-1),
                          's': tensor(states)})
        self.states = next_states
        self.total_steps += self.config.num_workers


    def learn(self):
        #--------------LEARNING------------------------#
        config = self.config
        storage = self.storage
        states = self.states
        
        prediction = self.network(states)
        storage.add(prediction)
        storage.placeholder()
        
        advantages = tensor(np.zeros((config.num_workers, 1)))
        returns = prediction['v'].detach()
        for i in reversed(range(config.rollout_length)):
            returns = storage.r[i] + config.discount * storage.m[i] * returns
            if not config.use_gae:
                advantages = returns - storage.v[i].detach()
            else:
                td_error = storage.r[i] + config.discount * storage.m[i] * storage.v[i + 1] - storage.v[i]
                advantages = advantages * config.gae_tau * config.discount * storage.m[i] + td_error
            storage.adv[i] = advantages.detach()
            storage.ret[i] = returns.detach()

        states, actions, log_probs_old, returns, advantages = storage.cat(['s', 'a', 'log_pi_a', 'ret', 'adv'])
        actions = actions.detach()
        log_probs_old = log_probs_old.detach()
        advantages = (advantages - advantages.mean()) / advantages.std()

        for _ in range(config.optimization_epochs):
            sampler = random_sample(np.arange(states.size(0)), config.mini_batch_size)
            for batch_indices in sampler:
                batch_indices = tensor(batch_indices).long()
                sampled_states = states[batch_indices]
                sampled_actions = actions[batch_indices]
                sampled_log_probs_old = log_probs_old[batch_indices]
                sampled_returns = returns[batch_indices]
                sampled_advantages = advantages[batch_indices]

                prediction = self.network(sampled_states, sampled_actions)
                ratio = (prediction['log_pi_a'] - sampled_log_probs_old).exp()
                obj = ratio * sampled_advantages
                obj_clipped = ratio.clamp(1.0 - self.config.ppo_ratio_clip,
                                          1.0 + self.config.ppo_ratio_clip) * sampled_advantages
                policy_loss = -torch.min(obj, obj_clipped).mean() - config.entropy_weight * prediction['ent'].mean()

                value_loss = 0.5 * (sampled_returns - prediction['v']).pow(2).mean()

                approx_kl = (sampled_log_probs_old - prediction['log_pi_a']).mean()
                if approx_kl <= 1.5 * config.target_kl:
                    self.actor_opt.zero_grad()
                    policy_loss.backward()
                    self.actor_opt.step()

                self.critic_opt.zero_grad()
                value_loss.backward()
                self.critic_opt.step()


### run_steps

In [86]:
#         for _ in range(config.rollout_length):
#             # step agent and environment

def run_steps(agent, scores_window_len=10, print_every=10):
    config = agent.config
    agent_name = agent.__class__.__name__
    t0 = time.time()

    scores_per_episode = []                                  # List containing agent scores from each episode
    scores_window = deque(maxlen=config.scores_window_len)   # last however many scores

    for episode in range(1, config.max_episodes+1):
        env_info = config.environment.reset(train_mode=True)[brain_name]    # reset the environment
        states = env_info.vector_observations                # get the current states
        agent.states = agent.config.state_normalizer(states) # set agent states to normalized states
        scores = np.zeros(config.num_agents)                 # initialize the score (for each agent)
        for t in range(1, config.max_steps+1):   
            agent.step()
            scores += agent.rewards                          # update the score (for each agent)
                                                             # display time steps and mean of all agent scores
            print('\rEpisode {}\tScore: {:.2f}\tTime Step {}'
                  .format(episode, np.mean(scores), t), end=" "*5)                 
            if np.any(agent.dones):                          # exit loop if episode finished
                break   
        if agent.total_steps % config.rollout_length == 0:   # Learn 
            agent.learn()
        scores_window.append(scores)                         # save most recent scores
        scores_per_episode.append(scores)                    # save most recent scores
        if episode % config.print_every == 0:                # output average of scores in print_every interval
            print('\rEpisode {}\tAverage Score: {:.2f}'
                  .format(episode, np.mean(scores_per_episode[-config.print_every:])), end=" "*35+"\n") 

### PPO Example

In [119]:
def ppo_continuous(**kwargs):
    kwargs.setdefault('log_level', 0)
    config = Config()
    config.merge(kwargs)

    # TODO: Delete line below
    # config.task_fn = lambda: Task(config.game)
    config.environment = env
    config.brain_name = brain_name
    config.state_dim = state_size
    config.action_dim = action_size
    config.num_agents = num_agents

    config.network_fn = lambda: GaussianActorCriticNet(
        config.state_dim, config.action_dim, actor_body=FCBody(config.state_dim, gate=torch.tanh),
        critic_body=FCBody(config.state_dim, gate=torch.tanh))
    config.actor_opt_fn = lambda params: torch.optim.Adam(params, 3e-4)
    config.critic_opt_fn = lambda params: torch.optim.Adam(params, 1e-3)
    config.discount = 0.99
    config.use_gae = False
    config.gae_tau = 0.95
    config.gradient_clip = 0.5
    config.max_episodes = 4000
    config.rollout_length = 2048
    config.optimization_epochs = 10
    config.mini_batch_size = 32
    config.ppo_ratio_clip = 0.2
    config.log_interval = 2048
    config.max_steps = int(1e4)
    config.target_kl = 0.01
    config.state_normalizer = MeanStdNormalizer()
    config.print_every = 100
    config.scores_window_len = 100
    ppo_agent = PPOAgent(config)
    run_steps(ppo_agent)
    return ppo_agent


### Simply train

In [120]:
agent = ppo_continuous()

Episode 100	Average Score: 0.01                                            
Episode 200	Average Score: 0.01                                    
Episode 300	Average Score: 0.01                                   
Episode 400	Average Score: 0.00                                         
Episode 500	Average Score: 0.00                                   
Episode 600	Average Score: 0.01                                         
Episode 700	Average Score: 0.01                                   
Episode 800	Average Score: 0.01                                                        
Episode 900	Average Score: 0.00                                    
Episode 1000	Average Score: 0.01                                                 
Episode 1100	Average Score: 0.00                                              
Episode 1200	Average Score: 0.01                                         
Episode 1300	Average Score: 0.01                                                   
Episode 1400	Average Score: 0.01  

In [96]:
agent.storage.m[i]

IndexError: list index out of range

## `RUN ALL ABOVE`

In [9]:
"""
Simply train
"""

#Create agents
agents = PPOAgent()
# agents = ddpg_agent.Agent(state_size, action_size, num_agents, RANDOM_SEED)
# agents.reset()                                             # reset the agent noise function to mu


for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)                                 
    t = 0                                                  # initialize number of timesteps
    while True:
        # select an action (for each agent)
        actions = agents.act(states)               
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        # Save the experience and learn
        agents.step(states, actions, rewards, next_states, dones)    
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        t+=1                                               # update number of time steps
#         print('\rTime Steps: {}'.format(t), end="")      
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('\nTotal score (averaged over agents) episode {}: {}'.format(np.mean(scores)))
    print('Average score: {}\tLength in time steps: {}'.format(np.mean(scores), t))
    

NameError: name 'PPO_agent' is not defined

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [11]:
# Modified from Continuous Control

def train_agent(n_episodes=175, max_t=1100, scores_window_len=100, goal_score = 30, 
         print_every = 10, save_every=10, resume_training=False, path = 'checkpoint.tar'):
    """Deep Q-Learning.
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        scores_window_len: a rolling window size for averaging the scores
        goal_score: the minimum average score required for a satisfactory model
        print_every: how often to print an average score for that interval 
            (average score is not for scores_window_len!)
        save_every: interval for saving the model weights, optimizers, and memory
        resume_training: whether or not to start training from a saved checkpoint
        path: filename for loading and saving the checkpoint
    """
    agents = ddpg_agent.Agent(state_size, action_size, num_agents, RANDOM_SEED)  # create agents
    
    scores_per_episode = []                                  # List containing agent scores from each episode
    time_steps_per_episode = []                              # List containing time_steps from each episode
    scores_window = deque(maxlen=scores_window_len)          # last however many scores
    
    # Load a checkpoint
    if resume_training:
        agents, scores_per_episode, scores_window, = load_checkpoint(agents, scores_per_episode,
                                                                            scores_window, path)  
    
    for i_episode in range(len(scores_per_episode)+1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]    # reset the environment
        states = env_info.vector_observations                # get the current states
        scores = np.zeros(num_agents)                        # initialize the score (for each agent)
        agents.reset()                                       # reset internal noise state to mu
        for t in range(1, max_t+1):          
            actions = agents.act(states)                     # agents act
            env_info = env.step(actions)[brain_name]         # step the environment
            next_states = env_info.vector_observations       # observe next state
            rewards = env_info.rewards                       # observe rewards
            dones = env_info.local_done                      # observe dones             
                                                             # save the experience and learn
            agents.step(states, actions, rewards, next_states, dones)
            states = next_states                             # roll over states to next time step
            scores += rewards                                # update the score (for each agent)
                                                             # display time steps and mean of all agent scores
            print('\rEpisode {}\tScore: {:.2f}\tTime Step {}\tSize of memory: {}'
                  .format(i_episode, np.mean(scores), t, len(agents.memory)), end=" "*5)
            if np.any(dones):                                # exit loop if episode finished
                break    

        scores_window.append(scores)                         # save most recent scores
        scores_per_episode.append(scores)                    # save most recent scores
        time_steps_per_episode.append(t)                     # save how many time steps the episode lasted
        if i_episode % print_every == 0:                     # output average of scores in print_every interval
            print('\rEpisode {}\tAverage Score: {:.2f}\tAverage time steps: {:.1f}'
                  .format(i_episode, 
                          np.mean(scores_per_episode[-print_every:]), 
                          np.mean(time_steps_per_episode[-print_every:])), 
                  end=" "*35+"\n")    
        if np.mean(scores_window)>=goal_score:               # proclaim the evironment solved, save checkpoints 
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'
                  .format(i_episode-scores_window_len, np.mean(scores_window)))
            save_checkpoint(agents, scores_per_episode, ddpg_agent, path)
            break
        if i_episode % save_every ==0:                        # save the models
            save_checkpoint(agents, scores_per_episode, ddpg_agent, path)
    return scores_per_episode, time_steps_per_episode

In [12]:
def run_agent(n_episodes=175, max_t=1001, scores_window_len=100, goal_score = 30, 
              print_every=10, save_every=10,
              resume_training=False, path='checkpoint.tar', plot=False, title='Scores'):
    """ Runs an agent over episodes and returns the scores from each episode  
        Attempts to make hyperparameter tuning easier
        Updates for the changes in the dqn_agent.py file 
        Relies on the ddpg() function and helper.py functions
        plot: whether or not to plot the scores
        title: plot title
    """
    
    # reload library after modifying Hyperparameters
    importlib.reload(ddpg_agent)

    # Generate Scores
    tic = time.time()
    print_hyperparameters(ddpg_agent)       
    scores, time_steps = train_agent(n_episodes, max_t, scores_window_len, goal_score, 
                              print_every, save_every, resume_training, path)
    toc = time.time()
    print("\nTime elapsed: {:.0f} seconds".format(toc-tic))

    # Plot Scores
    if plot == True:
        plot_scores(scores, rolling_window=scores_window_len, title=title)

    return scores, time_steps

In [ ]:
scores, time_steps = run_agent(n_episodes=2000, max_t=3000, scores_window_len=100, goal_score = 0.9, 
                               print_every=10, save_every=100,
                               resume_training=False, path='checkpoint.tar', 
                               plot=True, title='2 tennis players and update every 3 timesteps')

BUFFER_SIZE = 100000
BATCH_SIZE = 128
GAMMA = 0.99
TAU = 0.001
LR_ACTOR = 0.0003
LR_CRITIC = 0.001
WEIGHT_DECAY = 0.0
UPDATE_EVERY = 3
N_UPDATES = 20
FC1_UNITS_ACTOR = 256
FC2_UNITS_ACTOR = 128
FCS1_UNITS_CRITIC = 256
FC2_UNITS_CRITIC = 128
Episode 410	Average Score: 0.66	Average time steps: 269.5                                   
Episode 420	Average Score: 0.41	Average time steps: 173.8                                   
Episode 430	Average Score: 0.42	Average time steps: 180.8                                   
Episode 440	Average Score: 0.42	Average time steps: 176.6                                   
Episode 450	Average Score: 0.36	Average time steps: 150.0                                   
Episode 460	Average Score: 0.35	Average time steps: 149.8                                   
Episode 470	Average Score: 0.17	Average time steps: 80.7                                   
Episode 480	Average Score: 0.25	Average time steps: 113.4                                   
Episode 490	Aver

In [ ]:
def plot_scores(scores, rolling_window=10, title = 'Scores'):
    """ Plots the average score across agents for each episode and a rolling average"""
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), np.mean(scores, 1))
    rolling_mean = pd.Series(np.mean(scores,1)).rolling(rolling_window, 1).mean()
    plt.plot(rolling_mean)
    plt.title(title)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()
    return None